# * SNAP 의 Node2vec
- SNAP(Stanford Network Analysis Project) : http://snap.stanford.edu/node2vec/
- 논문 : https://arxiv.org/abs/1607.00653
- 구현 : c버젼과 python 버전 존재(python 버젼은 느려서 불가능 -> 코드 개선해야한다)

## * python 사용 대안
- 어떤분이 sparse matrix 로 구현하셨다 : https://www.singlelunch.com/2019/08/01/700x-faster-node2vec-models-fastest-random-walks-on-a-graph/
- 하지만 초기 adjency matrix 초기화시에 임의의 int로 elements를 채워놓는 문제로, graph edge가 존재하지 않는 경우에 문제가 있다('임의' int를 수정해야한다)
> https://github.com/VHRanger/graph2vec/blob/master/graph2vec/graph.py 내의 오류가 있다 
> - 58번째 line res = np.empty((n_walks, walklen), dtype=np.int64)<br/>
(이는 undirected일때 나타날 수 있는 문제 - directed라면 edge가 null인 node가 없을것이야)
> > - 대안1: undirected로 바꾼다

# * 오류 case는 가장 밑부분에 나와있다. 역추적해간다

---

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import glob
import os

%matplotlib inline

In [0]:
from sklearn import cluster, metrics

In [0]:
import networkx as nx
from collections import Counter

In [0]:
import pickle
import graph2vec # dir를 'graph2vec-master'로 맞춰주었잖아

---

## # load & using weighted Graph

In [ ]:
# 불러올때
with open ('./1107_with_cate/new_graph_list_w_cate_1115', 'rb') as fp:
    new_graph_list_ = pickle.load(fp)

In [0]:
import networkx as nx
from collections import Counter

In [0]:
#G = nx.Graph((x, y, {'weight': v}) for (x, y), v in Counter(node_edge).items()) # Undirected-graph

In [0]:
G_d = nx.DiGraph(new_graph_list_) # 만들어놓은 new_graph_list_ 활용

In [0]:
G_d = nx.DiGraph((x, y, {'weight': v}) for (x, y), v in Counter(node_edge).items()) # directed-graph

In [0]:
len(G_d)

1371788

In [0]:
len(G_d) # kwd로 node(상품) 100만개

1087231

In [0]:
len(G_d) # 5일동안 137만개의 상품이 있는겨 (물론 조금 filtering되었지만)

1371865

In [0]:
#nodes = list(G.nodes)
digraph_nodes = list(G_d.nodes)

In [0]:
#len(nodes)
len(digraph_nodes)

1371788

#### cf. undirected vs. directed 노드의 순서 같나? - 같다

In [0]:
# Undirected 에서
print(nodes[:5] , nodes[-5:])

[600181479, 600186873, 600185426, 600118584, 600186422] [101361801, 156809981, 157125654, 160981088, 152397171]


In [0]:
# directed 에서
print(nodelist_d[:5] , nodelist_d[-5:])

[600181479, 600186873, 600185426, 600118584, 600186422] [101361801, 156809981, 157125654, 160981088, 152397171]


In [0]:
nodes[:5] # directed

[600181479, 600186873, 600185426, 600118584, 600186422]

In [0]:
nodelist[:5] # directed 저장본 불러왔을때

[600181479, 600186873, 600185426, 600118584, 600186422]



---



---



### # Using node2vec

In [0]:
import multiprocessing

multiprocessing.cpu_count()

40

In [0]:
??graph2vec.Node2Vec() # 기본 파라미터를 참고하여 훈련하자 (w2vparams 값 하나만 바꾸면,  나머지 파라미터들은 word2vec의 기본 파라미터들이 적용되는 문제가 있는것 같다 - 100차원으로 훈련되는것에서 힌트)

### 기본파라미터
#walklen=10, epochs=20, return_weight=1., 
#        neighbor_weight=1., threads=0,
#        w2vparams={"window":10, "size":32, "negative":20, "iter":10,
#                   "batch_words":128})

In [0]:
## 기본 setting 에서  - walklen:5 , epochs:30, negative:5 만 바꿨다

n2v = graph2vec.Node2Vec(threads=0, walklen = 10, return_weight=1. , neighbor_weight=1. , epochs=15,  
                         w2vparams={'window':10, 'size':32, "negative": 20, "iter":10, "batch_words":128})

In [0]:
%pwd

'/content/gdrive/My Drive/Colab Notebooks/공부/embedding_model/Graph embedding/1029_graph2vec/graph2vec-master'

In [0]:
%%time 
# random_walk 따로 뽑아오고싶다
## 기본 - walklen=10, epochs=20, w2vparams={window:10, size=32, negative:20, iter:10, batch_words:128}
walks = n2v.fit_make_walks(G_d)

Making walks... Done, T=33.87
Mapping Walk Names... CPU times: user 26min 51s, sys: 20.7 s, total: 27min 12s
Wall time: 5min 56s


In [0]:
len(G_d.nodes)

1371788

In [0]:
type(walks)

list

In [0]:
len(walks)

20576820

In [0]:
len(G_d.nodes)*15 # epochs 만큼인거 확인!

20576820

In [0]:
walks[:5]

[['600181479',
  '4416758',
  '154787604',
  '153289529',
  '153150289',
  '154985086',
  '4417845',
  '4429859',
  '4428821',
  '161377353'],
 ['600186873',
  '160809230',
  '600011229',
  '100314905',
  '4398247',
  '1869895',
  '1874687',
  '131976415',
  '2158386',
  '3769103'],
 ['600185426',
  '2102732',
  '4341809',
  '4424216',
  '4425372',
  '4424216',
  '4284836',
  '4303445',
  '4385637',
  '4372804'],
 ['600118584',
  '4425637',
  '600204978',
  '4428342',
  '4428099',
  '4427650',
  '4428637',
  '4430570',
  '4427100',
  '600187245'],
 ['600186422',
  '160603299',
  '127613301',
  '152055767',
  '4324783',
  '3573618',
  '142461616',
  '153821757',
  '142290362',
  '142290320']]

----

### cf. normalize row랑, sparse.csr_matrix 에서 indptr 사용법 살피기

In [0]:
tmp = np.array([[0,2,0,1],
                [1,0,0,0],
                [0,0,0,0],
                [0,0,0,0]])

In [0]:
def _sparse_normalize_rows(mat):
    """
    Normalize a sparse CSR matrix row-wise (each row sums to 1)
    If a row is all 0's, it remains all 0's
    
    Parameters
    ----------
    mat : scipy.sparse.csr matrix
        Matrix in CSR sparse format
    Returns
    -------
    out : scipy.sparse.csr matrix
        Normalized matrix in CSR sparse format
    """
    n_nodes = mat.shape[0]
    # Normalize Adjacency matrix to transition matrix
    # Diagonal of the degree matrix is the sum of nonzero elements
    degrees_div = np.array(np.sum(mat, axis=1)).flatten()
    # This is equivalent to inverting the diag mat
    # weights are 1 / degree
    degrees = np.divide(
        1,
        degrees_div,
        out=np.zeros_like(degrees_div, dtype=float),
        where=(degrees_div != 0)
    )
    # construct sparse diag mat 
    # to broadcast weights to adj mat by dot product
    D = sparse.dia_matrix((n_nodes,n_nodes), dtype=np.float64)
    D.setdiag(degrees)   
    # premultiplying by diag mat is row-wise mul
    return sparse.csr_matrix(D.dot(mat))

In [0]:
T = _sparse_normalize_rows(A);T

<1371788x1371788 sparse matrix of type '<class 'numpy.float64'>'
	with 10496487 stored elements in Compressed Sparse Row format>

In [0]:
tmp_normalized = _sparse_normalize_rows(tmp);tmp_normalized

matrix([[0.        , 0.66666667, 0.        , 0.33333333],
        [1.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ]])

In [0]:
tmp_csr = sparse.csr_matrix(tmp_normalized)

In [0]:
tmp_csr.data

array([0.66666667, 0.33333333, 1.        ])

In [0]:
tmp_csr.indices

array([1, 3, 0], dtype=int32)

In [0]:
tmp_csr.indptr

array([0, 2, 3, 3, 3], dtype=int32)

In [0]:
sampling_nodes = [0,1]

In [0]:
n_walks = 2
walklen = 5
res = np.empty((n_walks, walklen), np.int64)

In [0]:
cdf_ = cdf[0]

In [0]:
cdf_.todense()

matrix([[0.        , 0.66666667, 0.        , 0.33333333],
        [1.        , 0.        , 0.        , 0.        ]])

In [0]:
print(p)

  (0, 1)	0.6666666666666666
  (0, 3)	0.3333333333333333
  (1, 0)	1.0


---

### # 문제가되는 3874398('와플수세미') 상품의 random-walk 조사해보자

In [0]:
walk_3874398 = []

for walk in walks:
    if '3874398' in walk:
        walk_3874398.append(walk) 

In [0]:
len(walk_3874398)# 106개가 포함

106

In [0]:
# 3874398은 계속 똑같은 node로 이동하네???
[walk for walk in walks if walk[0] == '3874398' ] # epochs를 15로 했더니, 맞네

[['3874398',
  '107057133',
  '101117477',
  '3357781',
  '135709355',
  '3357781',
  '2537289',
  '100515948',
  '600158489',
  '155175570'],
 ['3874398',
  '107057133',
  '600051152',
  '600055677',
  '600017925',
  '2974502',
  '600062355',
  '600090377',
  '600041542',
  '600057126'],
 ['3874398',
  '107057133',
  '112622009',
  '122018726',
  '147674953',
  '122018726',
  '147674953',
  '122018726',
  '122018717',
  '163153072'],
 ['3874398',
  '107057133',
  '2389354',
  '600114667',
  '4424322',
  '4424858',
  '4424622',
  '4425107',
  '4415307',
  '4421273'],
 ['3874398',
  '107057133',
  '600051152',
  '600055677',
  '1578142',
  '600098202',
  '124265882',
  '120031505',
  '600055677',
  '600098202'],
 ['3874398',
  '107057133',
  '101117477',
  '123466405',
  '123466480',
  '4426586',
  '4425915',
  '4426339',
  '4426883',
  '4426430'],
 ['3874398',
  '107057133',
  '2389354',
  '600013401',
  '600114667',
  '2300524',
  '3122676',
  '3521291',
  '4412029',
  '2853745'],
 ['

In [0]:
# adjency matrix 조사
A = nx.adj_matrix(G_d)

In [0]:
A.shape

(1371788, 1371788)

In [0]:
# nodes를 리스트형태로
directed_nodes = list(G_d.nodes())

In [0]:
len(directed_nodes)

1371788

In [0]:
directed_nodes[:5]

[600181479, 600186873, 600185426, 600118584, 600186422]

In [0]:
# directed_nodes에서 3874398 를 찾아보자(결과 index:359464)

for i,node in enumerate(directed_nodes):
    if node == 3874398:
        print('index :', i, ', node :', node)

index : 359464 , node : 3874398


----

### * 해당 index의 random-walk 과정 살펴보자(오류 찾았따!)

In [0]:
import numba

In [0]:
Tdata = T.data
Tindptr = T.indptr
Tindices = T.indices
walklen = 10

sampling_nodes = [359464,359463]

n_walks = len(sampling_nodes)
res = np.empty((n_walks, walklen), dtype=np.int64)

for i in numba.prange(n_walks):
    state = sampling_nodes[i]
    for k in range(walklen-1):
        res[i, k] = state
        start = Tindptr[state]
        end = Tindptr[state+1]
        print('res[{},{}]:'.format(i,k), state, 'start:',start, 'end:',end)
        # transition probabilities
        p = Tdata[start:end]
        print('p : {}'.format(p))
        # cumulative distribution of transition probabilities
        cdf = np.cumsum(p) # [0] 붙어야하나?
        print('cdf : {}'.format(cdf))
        # Random draw in [0, 1] for each row
        # Choice is where random draw falls in cumulative distribution
        draw = np.random.rand()
        print('draw : {}'.format(draw))
        
        # Find where draw is in cdf
        # Then use its index to update state
        next_idx = np.searchsorted(cdf, draw)
        print('next_idx : {}'.format(next_idx))
        
        # Winner points to the column index of the next node
        state = Tindices[start + next_idx]
        # Write final states
        print('\n')
    res[i, -1] = state
    print('res[{},-1]:'.format(i), res[i, -1])
    print('------------------------------------------------')
    print('\n')

res[0,0]: 359464 start: 8709785 end: 8709785
p : []
cdf : []
draw : 0.8660822452444411
next_idx : 0


res[0,1]: 103158 start: 6582194 end: 6582205
p : [0.10989011 0.06593407 0.06593407 0.06593407 0.10989011 0.06593407
 0.01098901 0.10989011 0.06593407 0.10989011 0.21978022]
cdf : [0.10989011 0.17582418 0.24175824 0.30769231 0.41758242 0.48351648
 0.49450549 0.6043956  0.67032967 0.78021978 1.        ]
draw : 0.8126232044264129
next_idx : 10


res[0,2]: 58195 start: 5577438 end: 5577449
p : [0.07692308 0.07692308 0.04615385 0.07692308 0.09230769 0.15384615
 0.07692308 0.07692308 0.04615385 0.23076923 0.04615385]
cdf : [0.07692308 0.15384615 0.2        0.27692308 0.36923077 0.52307692
 0.6        0.67692308 0.72307692 0.95384615 1.        ]
draw : 0.5199250918663558
next_idx : 5


res[0,3]: 91161 start: 6363075 end: 6363166
p : [0.00347021 0.00578369 0.00578369 0.00578369 0.00578369 0.00347021
 0.00578369 0.00578369 0.00578369 0.00578369 0.00578369 0.00578369
 0.00578369 0.00578369 0.003

In [0]:
res

array([[ 359464,  103158,   58195,   91161,   82452,   40305,   62204,
        1028831,   46243,  131561],
       [ 359463,  359464,  103158,  474508,  639016,   19135,   91857,
         110427,   47964,   14460]])

* **오류가 있다 : 분명 첫번째 random-walk에서 갈곳이 없었는데, 이상한곳으로 mapping 된다!**

---

### # node2vec fitting

In [0]:
%%time 
## 기본 - walklen=10, epochs=20, w2vparams={window:10, size=32, negative:20, iter:10, batch_words:128}
n2v.fit(G_d) 


embed_size = 32 # Node2Vec의 기본 embed-size 는 32였어
embeds = np.empty((len(digraph_nodes), embed_size))

for i,node in enumerate(digraph_nodes):
    embeds[i] = n2v.predict(node)

np.save('1113_direct_base.npy', embeds)

Making walks... Done, T=29.60
Mapping Walk Names... Done, T=246.09
Training W2V... Done, T=8830.81


/content/gdrive/My Drive/Colab Notebooks/공부/embedding_model/Graph embedding/1029_graph2vec/graph2vec-master/graph2vec/graph.py:406: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return self.model[node_name]


CPU times: user 6h 23min 46s, sys: 33min 55s, total: 6h 57min 42s
Wall time: 2h 33min 5s


In [0]:
embeds

array([[ 1.22905171, -3.26411247,  2.05514336, ..., -3.58916736,
         2.38996267,  2.13926458],
       [-0.53847474, -5.56275606,  1.86666739, ..., -0.13200162,
         5.59156132, -0.04875129],
       [-0.1989885 , -1.45533049, -0.08335733, ..., -0.0689493 ,
         2.78106451, -5.79675436],
       ...,
       [ 0.25762832,  0.19774814,  0.93224514, ...,  0.27430385,
         0.54954588, -2.33664894],
       [ 0.00925384, -0.3487694 ,  0.28238174, ..., -0.28226861,
         0.06169181, -0.42875022],
       [-0.05393747, -0.69453079,  1.14673924, ..., -0.14792757,
         0.61898339, -2.71857667]])

In [0]:
embeds.shape

(1371788, 32)

In [0]:
embeds

array([[ 4.48070955,  1.61329055, -1.55646265, ..., -1.68655717,
         0.71059108, -1.32855093],
       [ 0.84004813, -1.3062377 , -0.60076034, ..., -5.26315022,
         1.66197932,  0.73398423],
       [ 7.51180553, -1.19695747, -3.03270149, ..., -0.23695295,
         2.95918536, -2.91803145],
       ...,
       [ 0.98433661,  0.24287279,  0.27622041, ...,  0.97671133,
         0.98167813,  0.29304436],
       [-1.30843747, -0.89789724,  1.07976973, ...,  1.15732777,
        -0.04601382, -1.40657759],
       [-0.96856052, -0.27465129,  0.92876369, ...,  0.835365  ,
         0.0953926 , -0.85234773]])



---



---



### # embedding load하여 사용

In [0]:
#undirected = np.load('embedding_1029.npy')
#directed = np.load('./1104_kwd/di_embedding_1104_walk10_window10_ep20.npy') #
directed = embeds # 위의 훈련 결과 바로 쓸때 

In [0]:
directed.shape # 그냥 클릭 : 170만 -> kwd클릭 : 100만 -> 클릭_w_cate : 137만

(1371788, 32)

In [0]:
len(digraph_nodes)

1371788

In [0]:
digraph_nodes # embedding 의 순서는 이 순서야!

[600181479,
 600186873,
 600185426,
 600118584,
 600186422,
 158200378,
 151723408,
 4241527,
 4421573,
 148705230,
 600006367,
 143699050,
 160485156,
 152811662,
 157231307,
 152069791,
 4407271,
 4421078,
 4421789,
 600183527,
 4422021,
 4422034,
 4423136,
 4423093,
 600183568,
 4340713,
 4344722,
 4425136,
 4424258,
 4425152,
 4424659,
 4414577,
 600187942,
 159173216,
 153543307,
 160960557,
 600178861,
 155399929,
 160602340,
 600172878,
 158923084,
 157683844,
 600183029,
 161377353,
 4423142,
 4425677,
 4424670,
 600114430,
 4418502,
 4415306,
 4403966,
 4390047,
 4420571,
 600227607,
 4428842,
 600156346,
 4428511,
 600183689,
 4413772,
 4337134,
 158088582,
 4412226,
 142988306,
 122364967,
 104077238,
 132874992,
 1308238,
 2497111,
 2658930,
 600056265,
 600186059,
 160948238,
 600025810,
 101441562,
 145610838,
 162772363,
 162772258,
 162769548,
 162768208,
 162762304,
 161789925,
 161370250,
 161369764,
 156108781,
 156102641,
 155340404,
 155265956,
 155082051,
 1544948

## # embedding 밝혀준다
- embedding 어떤 순서
- did, d_nm 명시

In [0]:
print( digraph_nodes[:5] , digraph_nodes[-5:] )

[600181479, 600186873, 600185426, 600118584, 600186422] [101361801, 156809981, 157125654, 160981088, 152397171]


In [0]:
glob.glob('../../../unique_deal_name/*')

['../../../unique_deal_name/part-00000-077ffd9c-243a-4192-b5ca-b1765355727c-c000.csv']

In [0]:
# 여기선 category 정보 입힌 딜정보 사용
unique_deal_name = deal_w_cate \
  = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/공부/embedding_model/Graph embedding/1029_graph2vec/graph2vec-master/1107_with_cate/deal_w_cate_1108.csv', index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
unique_deal_name.head()

,did,d_nm,cate,cate_nm
0,273551,[97무배]닥터수 피부관리샵 모델링팩,234020105,234020105_팩/마스크
1,3651575,고유에 프로덤 GOLD SILK 마스크,234020105,234020105_팩/마스크
2,3922380,[원더쿠폰] 다이아필오프팩+사은 2종,234020105,234020105_팩/마스크
3,3369862,[무료배송] 여드름 스팟 트러블패치,234020105,234020105_팩/마스크
4,3386562,남해청정해역 생다시마를 담은,234020105,234020105_팩/마스크


In [0]:
unique_deal_name.shape

(1457035, 4)

In [0]:
pd.DataFrame({'did':digraph_nodes}).merge(unique_deal_name, how='left').shape

(1371788, 4)

In [0]:
pd.DataFrame({'did':digraph_nodes}).merge(unique_deal_name, how='left').head()

,did,d_nm,cate,cate_nm
0,600181479,[만반잘부] 오바다 부드러운 국내산 오징어,4102681,4102681_건오징어/반건오징어
1,600186873,[만반잘부] 주재원 삼겹살/오겹살/목살 500g 모음전,4102757,4102757_삼겹살/오겹살
2,600185426,핫한 청도 아이스홍시 & 감말랭이 모음전,4102648,4102648_아이스홍시
3,600118584,사과즙/도라지배즙/배도라지즙/포도즙/배즙/통째로 과일즙,4104154,4104154_유기농/친환경주스
4,600186422,악세사리함 쥬얼리함 보석함 시계보관함 모음전,4104556,4104556_쥬얼리 용품/세척


In [0]:
pd.DataFrame({'did':digraph_nodes}).merge(unique_deal_name, how='left').tail()

,did,d_nm,cate,cate_nm
1371783,101361801,[무료배송]키친아트 아르떼 열센서 후라이팬 30cm 인덕션겸용,4103720,4103720_프라이팬 낱개
1371784,156809981,인증점K 제트 VS20R9044SA 무선청소기 미세먼지차단 베터리패키지,4100558,4100558_삼성
1371785,157125654,[바보사랑]애플펜슬2세대 듀오 실리콘 케이스,4101328,4101328_이외 스마트폰 액세서리
1371786,160981088,화분월드동그라미 화분받침 2호,4103919,4103919_화분
1371787,152397171,식품/생활/유아동,4106318,4106318_비즈닛


In [0]:
unique_deal_name_ = pd.DataFrame({'did':digraph_nodes}).merge(unique_deal_name, how='left').drop(columns=['cate', 'cate_nm'])

In [0]:
unique_deal_name_.head()

,did,d_nm
0,600181479,[만반잘부] 오바다 부드러운 국내산 오징어
1,600186873,[만반잘부] 주재원 삼겹살/오겹살/목살 500g 모음전
2,600185426,핫한 청도 아이스홍시 & 감말랭이 모음전
3,600118584,사과즙/도라지배즙/배도라지즙/포도즙/배즙/통째로 과일즙
4,600186422,악세사리함 쥬얼리함 보석함 시계보관함 모음전


---
### 일단 합치고, 상품 정보 없는거 날려주어야겠다 - gensim 에서 similar 못보잖아 (nan이 중복되게 많으니깐)

In [0]:
pd.concat([unique_deal_name_, pd.DataFrame(directed)], axis=1).dropna().shape

(1371788, 34)

In [0]:
pd.concat([unique_deal_name_, pd.DataFrame(directed)], axis=1).head()

,did,d_nm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,600181479,[만반잘부] 오바다 부드러운 국내산 오징어,1.229052,-3.264112,2.055143,1.580373,-1.305503,-0.773409,4.613352,1.593006,-3.262018,-2.191530,1.096127,1.941396,1.369102,-2.222415,0.654294,-4.260469,-0.702936,4.347555,-0.135871,-4.510448,-2.119558,0.390893,3.758309,1.867299,-4.477293,-0.084679,3.284910,0.624318,-1.526585,-3.589167,2.389963,2.139265
1,600186873,[만반잘부] 주재원 삼겹살/오겹살/목살 500g 모음전,-0.538475,-5.562756,1.866667,0.805510,-0.860459,0.686097,-0.561745,-1.243336,0.387312,1.979101,0.278067,0.867419,-0.660043,-0.967203,-1.415900,-0.279134,1.713486,1.063283,0.232399,-2.199799,-2.226140,2.356587,-0.654963,3.594916,-2.185772,-0.066778,2.100049,-2.355687,0.481627,-0.132002,5.591561,-0.048751
2,600185426,핫한 청도 아이스홍시 & 감말랭이 모음전,-0.198988,-1.455330,-0.083357,2.721674,1.698118,6.046679,2.205015,-1.031879,-3.351793,-1.754357,0.923496,-0.607799,-0.060533,-0.039705,2.217461,-5.524176,-0.236714,-0.811096,-0.219093,-0.468020,1.993686,-3.131876,-2.162918,7.130883,0.283335,1.802665,4.820302,-2.781886,0.191043,-0.068949,2.781065,-5.796754
3,600118584,사과즙/도라지배즙/배도라지즙/포도즙/배즙/통째로 과일즙,-0.826809,0.229932,2.354365,4.996829,-3.060762,-1.269347,-0.884455,-5.253204,-2.990948,-0.807123,-1.994426,-3.365553,1.850344,-4.325764,-4.673232,-1.093071,0.116866,-0.506159,0.365160,3.885775,2.592640,-1.485418,-0.874233,3.603109,-4.192461,0.198641,-0.952814,-3.046672,0.678599,-4.625407,2.105101,-2.815536
4,600186422,악세사리함 쥬얼리함 보석함 시계보관함 모음전,-0.876567,0.342568,1.907246,-1.139385,-0.793353,-0.952386,0.132055,1.211387,-2.625733,2.470055,-1.640479,-0.200465,-1.838699,-2.038641,-0.368154,1.752448,0.361291,-2.492242,0.482906,1.329928,-3.390306,-2.432199,-1.787804,0.641939,1.249624,-0.763985,-1.409229,-0.167177,2.549449,-1.421691,1.078133,-1.925694


In [0]:
word_embed = pd.concat([unique_deal_name_, pd.DataFrame(directed)], axis=1)

In [0]:
word_embed.head()

,did,d_nm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,600181479,[만반잘부] 오바다 부드러운 국내산 오징어,1.229052,-3.264112,2.055143,1.580373,-1.305503,-0.773409,4.613352,1.593006,-3.262018,-2.191530,1.096127,1.941396,1.369102,-2.222415,0.654294,-4.260469,-0.702936,4.347555,-0.135871,-4.510448,-2.119558,0.390893,3.758309,1.867299,-4.477293,-0.084679,3.284910,0.624318,-1.526585,-3.589167,2.389963,2.139265
1,600186873,[만반잘부] 주재원 삼겹살/오겹살/목살 500g 모음전,-0.538475,-5.562756,1.866667,0.805510,-0.860459,0.686097,-0.561745,-1.243336,0.387312,1.979101,0.278067,0.867419,-0.660043,-0.967203,-1.415900,-0.279134,1.713486,1.063283,0.232399,-2.199799,-2.226140,2.356587,-0.654963,3.594916,-2.185772,-0.066778,2.100049,-2.355687,0.481627,-0.132002,5.591561,-0.048751
2,600185426,핫한 청도 아이스홍시 & 감말랭이 모음전,-0.198988,-1.455330,-0.083357,2.721674,1.698118,6.046679,2.205015,-1.031879,-3.351793,-1.754357,0.923496,-0.607799,-0.060533,-0.039705,2.217461,-5.524176,-0.236714,-0.811096,-0.219093,-0.468020,1.993686,-3.131876,-2.162918,7.130883,0.283335,1.802665,4.820302,-2.781886,0.191043,-0.068949,2.781065,-5.796754
3,600118584,사과즙/도라지배즙/배도라지즙/포도즙/배즙/통째로 과일즙,-0.826809,0.229932,2.354365,4.996829,-3.060762,-1.269347,-0.884455,-5.253204,-2.990948,-0.807123,-1.994426,-3.365553,1.850344,-4.325764,-4.673232,-1.093071,0.116866,-0.506159,0.365160,3.885775,2.592640,-1.485418,-0.874233,3.603109,-4.192461,0.198641,-0.952814,-3.046672,0.678599,-4.625407,2.105101,-2.815536
4,600186422,악세사리함 쥬얼리함 보석함 시계보관함 모음전,-0.876567,0.342568,1.907246,-1.139385,-0.793353,-0.952386,0.132055,1.211387,-2.625733,2.470055,-1.640479,-0.200465,-1.838699,-2.038641,-0.368154,1.752448,0.361291,-2.492242,0.482906,1.329928,-3.390306,-2.432199,-1.787804,0.641939,1.249624,-0.763985,-1.409229,-0.167177,2.549449,-1.421691,1.078133,-1.925694


In [0]:
word_embed.values.shape

(577798, 34)

In [0]:
word_embed.to_csv('./1107_with_cate/word_embed_1113_direct(base).csv')

### # word_embed_direct.csv 불러와서 쓴다!

In [0]:
#word_embed_direct = pd.read_csv('./word_embed_direct.csv')
word_embed_direct = word_embed # 지금은 그냥 위에 훈련한거 바로 쓸게

In [0]:
# did 필요없고
word_embed_direct.iloc[:,1:].head()


,d_nm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,[만반잘부] 오바다 부드러운 국내산 오징어,1.229052,-3.264112,2.055143,1.580373,-1.305503,-0.773409,4.613352,1.593006,-3.262018,-2.191530,1.096127,1.941396,1.369102,-2.222415,0.654294,-4.260469,-0.702936,4.347555,-0.135871,-4.510448,-2.119558,0.390893,3.758309,1.867299,-4.477293,-0.084679,3.284910,0.624318,-1.526585,-3.589167,2.389963,2.139265
1,[만반잘부] 주재원 삼겹살/오겹살/목살 500g 모음전,-0.538475,-5.562756,1.866667,0.805510,-0.860459,0.686097,-0.561745,-1.243336,0.387312,1.979101,0.278067,0.867419,-0.660043,-0.967203,-1.415900,-0.279134,1.713486,1.063283,0.232399,-2.199799,-2.226140,2.356587,-0.654963,3.594916,-2.185772,-0.066778,2.100049,-2.355687,0.481627,-0.132002,5.591561,-0.048751
2,핫한 청도 아이스홍시 & 감말랭이 모음전,-0.198988,-1.455330,-0.083357,2.721674,1.698118,6.046679,2.205015,-1.031879,-3.351793,-1.754357,0.923496,-0.607799,-0.060533,-0.039705,2.217461,-5.524176,-0.236714,-0.811096,-0.219093,-0.468020,1.993686,-3.131876,-2.162918,7.130883,0.283335,1.802665,4.820302,-2.781886,0.191043,-0.068949,2.781065,-5.796754
3,사과즙/도라지배즙/배도라지즙/포도즙/배즙/통째로 과일즙,-0.826809,0.229932,2.354365,4.996829,-3.060762,-1.269347,-0.884455,-5.253204,-2.990948,-0.807123,-1.994426,-3.365553,1.850344,-4.325764,-4.673232,-1.093071,0.116866,-0.506159,0.365160,3.885775,2.592640,-1.485418,-0.874233,3.603109,-4.192461,0.198641,-0.952814,-3.046672,0.678599,-4.625407,2.105101,-2.815536
4,악세사리함 쥬얼리함 보석함 시계보관함 모음전,-0.876567,0.342568,1.907246,-1.139385,-0.793353,-0.952386,0.132055,1.211387,-2.625733,2.470055,-1.640479,-0.200465,-1.838699,-2.038641,-0.368154,1.752448,0.361291,-2.492242,0.482906,1.329928,-3.390306,-2.432199,-1.787804,0.641939,1.249624,-0.763985,-1.409229,-0.167177,2.549449,-1.421691,1.078133,-1.925694


In [0]:
word_embed_direct.iloc[:,2:].shape

(1371788, 32)

### # word2vec용 txt파일 만들기(pretrained model)

In [0]:
word2vec_fmt_directed = [[1371788, 32]]

In [0]:
word2vec_fmt_directed

[[1371788, 32]]

In [0]:
### 상품명 공백 제거해줬다
word_embed_direct['d_nm'] = word_embed_direct.d_nm.apply(lambda x: "".join(x.split()))

In [0]:
word_embed_direct.head()

,did,d_nm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,600181479,[만반잘부]오바다부드러운국내산오징어,1.229052,-3.264112,2.055143,1.580373,-1.305503,-0.773409,4.613352,1.593006,-3.262018,-2.191530,1.096127,1.941396,1.369102,-2.222415,0.654294,-4.260469,-0.702936,4.347555,-0.135871,-4.510448,-2.119558,0.390893,3.758309,1.867299,-4.477293,-0.084679,3.284910,0.624318,-1.526585,-3.589167,2.389963,2.139265
1,600186873,[만반잘부]주재원삼겹살/오겹살/목살500g모음전,-0.538475,-5.562756,1.866667,0.805510,-0.860459,0.686097,-0.561745,-1.243336,0.387312,1.979101,0.278067,0.867419,-0.660043,-0.967203,-1.415900,-0.279134,1.713486,1.063283,0.232399,-2.199799,-2.226140,2.356587,-0.654963,3.594916,-2.185772,-0.066778,2.100049,-2.355687,0.481627,-0.132002,5.591561,-0.048751
2,600185426,핫한청도아이스홍시&감말랭이모음전,-0.198988,-1.455330,-0.083357,2.721674,1.698118,6.046679,2.205015,-1.031879,-3.351793,-1.754357,0.923496,-0.607799,-0.060533,-0.039705,2.217461,-5.524176,-0.236714,-0.811096,-0.219093,-0.468020,1.993686,-3.131876,-2.162918,7.130883,0.283335,1.802665,4.820302,-2.781886,0.191043,-0.068949,2.781065,-5.796754
3,600118584,사과즙/도라지배즙/배도라지즙/포도즙/배즙/통째로과일즙,-0.826809,0.229932,2.354365,4.996829,-3.060762,-1.269347,-0.884455,-5.253204,-2.990948,-0.807123,-1.994426,-3.365553,1.850344,-4.325764,-4.673232,-1.093071,0.116866,-0.506159,0.365160,3.885775,2.592640,-1.485418,-0.874233,3.603109,-4.192461,0.198641,-0.952814,-3.046672,0.678599,-4.625407,2.105101,-2.815536
4,600186422,악세사리함쥬얼리함보석함시계보관함모음전,-0.876567,0.342568,1.907246,-1.139385,-0.793353,-0.952386,0.132055,1.211387,-2.625733,2.470055,-1.640479,-0.200465,-1.838699,-2.038641,-0.368154,1.752448,0.361291,-2.492242,0.482906,1.329928,-3.390306,-2.432199,-1.787804,0.641939,1.249624,-0.763985,-1.409229,-0.167177,2.549449,-1.421691,1.078133,-1.925694


In [0]:
word2vec_fmt_directed.extend(word_embed_direct.iloc[:,1:].values.tolist())

In [0]:
len(word2vec_fmt_directed)

1371789

In [0]:
word2vec_fmt_directed[:2]

[[1371788, 32],
 ['[만반잘부]오바다부드러운국내산오징어',
  1.2290517091751099,
  -3.2641124725341797,
  2.055143356323242,
  1.5803725719451904,
  -1.3055033683776855,
  -0.7734090685844421,
  4.613351821899414,
  1.5930055379867554,
  -3.2620177268981934,
  -2.191530227661133,
  1.0961267948150635,
  1.9413959980010986,
  1.3691022396087646,
  -2.222414970397949,
  0.6542935967445374,
  -4.26046895980835,
  -0.7029355764389038,
  4.347554683685303,
  -0.1358708143234253,
  -4.510448455810547,
  -2.1195578575134277,
  0.39089319109916687,
  3.7583093643188477,
  1.8672987222671509,
  -4.477293014526367,
  -0.08467933535575867,
  3.284909963607788,
  0.6243177056312561,
  -1.5265854597091675,
  -3.589167356491089,
  2.389962673187256,
  2.1392645835876465]]

In [0]:
# save
with open('./1107_with_cate/1113_word2vec_fmt_cate_directed(base).txt', 'w') as f:
    for item in word2vec_fmt_directed:
        for k,element in enumerate(item):
            last = len(item)
            if k == last:
                f.write("%s" % element) # 마지막 요소는, 띄어쓰기 없다
            else:
                f.write("%s " % element)
            f.write("\n") # 한줄이 끝나면 줄 바꿈

In [0]:
## 이용해보자 gensim.models.Word2Vec.laod_word2vec_format

In [0]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./1107_with_cate/1113_word2vec_fmt_cate_directed(base).txt')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL




---





In [0]:
model.wv.most_similar( '림라이프피마자오일100ml' )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('피마자오일100ml1+1+1', 0.8831630349159241),
 ('피마자오일100ml피마자유캐스터오일속눈썹영양제탈모두피오일바디오일', 0.8599170446395874),
 ('보르헤스천연피마자유(비정제)1L피마자오일캐스터오일속눈썹영양제탈모두피오일', 0.8261103630065918),
 ('유앤오일/건성+노화+탄력너리시에너자이즈메이처페이스오일50ml', 0.7810809016227722),
 ('[더위치]피마자오일500ml/캐스태르오일/CastorO', 0.7714093327522278),
 ('정제피마자유100ml/500ml', 0.7642123699188232),
 ('[더위치]피마자오일100ml/캐스태르오일/Castor', 0.7640499472618103),
 ('보르헤스천연피마자유(비정제)500ml피마자오일캐스터오일속눈썹영양제탈모두피오일', 0.7593515515327454),
 ('[아이오페]골든글로우페이스오일40ml', 0.742207407951355),
 ('비정제피마자유아주까리기름', 0.7323443293571472)]

In [0]:
model.wv.most_similar( '[원더쿠폰]일회용와플수세미' ) ### 왜 이런 결과가????

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('부산명물부산어포포포500g대용량', 0.9689480066299438),
 ('[더반찬]삼색나물과고추장(3~4회분)', 0.9098036289215088),
 ('[바이홈]바른생활뽑아쓰는데일리수세미(30매입X4+2박스)', 0.8497264981269836),
 ('대용량부산어포포포500g', 0.847880482673645),
 ('대용량월드컵어포500g', 0.8291774988174438),
 ('부산어포포포대용량500g', 0.8202527761459351),
 ('[무료배송]대용량부산어포포포500g', 0.8106222152709961),
 ('추억의옛날그맛월드컵어포500g대용량', 0.8088663220405579),
 ('어포/꾸이킹/한진식품추가배송료없이전부구매가능', 0.7773061394691467),
 ('[무료배송]부산어포포포어채500g', 0.7496709823608398)]